# Distributed Databases and Big Data

- Rajath Akshay Vanikul

### Background
StopFire is a campaign started by Monash University to predict and stop the fire in Victorian cities. They have employed sensors in different cities of Victoria and have collected a large amount of data. The data is so big that their techniques have failed to provide the results on time to predict fire. We are asked to migrate their data to the NoSQL database (MongoDB).We also analyse the data to provide them with results. In addition, we will build an application, a complete setup from streaming to storing and analyzing the data for them using Apache Kafka, Apache Spark Streaming and MongoDB.

## Task A
## MongoDB Data Model 

Based on the two data sets provided i.e. `hotspot_historic.csv` and `climate_historic.csv`, we have designed a suitable data model to support the efficient storing and querying of the two data sets in MongoDB.

We know that the climate and fire data will be gathered in large quantities over the years whcih makes us think about a model which is easy to scale and yet have a distinguised event entities among different collections in database. We will be using model similar to [Anchor Modeling](https://en.wikipedia.org/wiki/Anchor_modeling)(reference model) but modify it according to our requirement.

Our data model adresses the following:
* high scalability.
* avoid null values.
* keep the information free from redundancies.
* being able to non-destructively evolve the model.

However, the only downside of our data model is that it increases the number of join query operations. We believes that analysis and query in a climate and hotspot data are performed only based on a small section of fields. This is similar to the array storage structure, which can significantly reduce data scanning and reduce the impact on query performance. Mongo DB is one of those databases with the table elimination feature that can greatly reduce the number of join operations.

In [2]:
# import the following libraries to perform this task.
from pymongo import MongoClient # access mongoDb using python.
import datetime 
import pandas as pd
from datetime import datetime
import dateutil.parser
import pymongo

In [3]:
# instantiating mongo client.
myclient = pymongo.MongoClient("mongodb://localhost:27017/")

# instantiating the database
mydb = myclient["assignment2"]

In [4]:
# instantiating the collections in the created database.
climate_collection = mydb["climate"] # collection named "climate"
fire_collection = mydb["fire"] # collection names "fire"

### schema creation
Creating the schema to validate the structure and declare the datatype of each entry.
Below is the code that addresses the schema for climate and fire collections in the data base.

In [5]:
# schema to validate the climate collection.
validator_schema_climate = {"$jsonSchema":
  {
         "bsonType": "object",
         "required": [ "station", "date", "air_temperature_celcius", "relative_humidity","windspeed_knots","max_wind_speed","precipitation"],
         "properties": {
            "station": {
               "bsonType": "int",
               "description": "must be an integer and is required"
            },
            "date": {
               "bsonType": "timestamp",
               "description": "must be a date and is not required"
            },
            "air_temperature_celcius": {
               "bsonType": "int",
               "description": "must be an integer and is required"
            },
            "relative_humidity": {
               "bsonType": "double",
               "description": "can only be double and is required"
            },
            "windspeed_knots": {
               "bsonType": ["double"],
               "description": "must be a double and is required"
            },
             "max_wind_speed": {
               "bsonType": [ "double" ],
               "description": "must be a double and is required"
            },
             "precipitation ": {
               "bsonType": "object",
               "description": "must be a string and is required"
            }
         }
  }
}

In [6]:
# schema to validate the fire collection.
validator_schema_fire = {"$jsonSchema":
  {
         "bsonType": "object",
         "required": [ "latitude", "longitude", "surface_temperature_celcius", "confidence","datetime","date"],
         "properties": {
            "latitude": {
               "bsonType": "double",
               "description": "must be an integer and is required"
            },
            "datetime": {
               "bsonType": "timestamp",
               "description": "must be a date and is not required"
            },
            "date": {
               "bsonType": "timestamp",
               "description": "must be a date and is not required"
            },
            "longitude": {
               "bsonType": "double",
               "description": "must be an integer and is required"
            },
            "surface_temperature_celcius": {
               "bsonType": "int",
               "description": "can only be int and is required"
            },
            "confidence": {
               "bsonType": "int",
               "description": "must be a int and is required"
            }
         }
  }
}

**Example for climate data model:**

`    
{
    '_id': ObjectId('5ce7568aa90b9bcb3df7ffb4'),
    'station': 948700,
    'date': datetime.datetime(2016, 12, 31, 0, 0),
    'air_temperature_celcius': 19,
    'relative_humidity': 56.8,
    'windspeed_knots': 7.9,
    'max_wind_speed': 11.1,
    'precipitation ': ' 0.00I'
}
`



**Example of fire data model:**

`
{
    '_id': ObjectId('5ce7568ba90b9bcb3df80122'),
	'latitude': -37.966,
	'longitude': 145.05100000000002,
	'datetime': datetime.datetime(2017, 12, 27, 4, 16, 51),
	'confidence': 78,
	'date': datetime.datetime(2017, 12, 27, 0, 0),
	'surface_temperature_celcius': 68
}`

## Task B

## Loading the historic data.

As the task suggests, we are loading our historic data into a NoSQL database(MongoDB) for the company.

In [19]:
# read the csv file into respective variables.
climate = pd.read_csv("climate_historic.csv")
fire = pd.read_csv("hotspot_historic.csv")

In [20]:
print("Shape of Climate data:",climate.shape)
print("Shape of Fire data:",fire.shape)

Shape of Climate data: (366, 7)
Shape of Fire data: (2668, 6)


In [21]:
# inspecting the climate data set.
climate.head(5)

,station,date,air_temperature_celcius,relative_humidity,windspeed_knots,max_wind_speed,precipitation
0,948700,31/12/2016,19,56.8,7.9,11.1,0.00I
1,948700,2/01/2017,15,50.7,9.2,13.0,0.02G
2,948700,3/01/2017,16,53.6,8.1,15.0,0.00G
3,948700,4/01/2017,24,61.6,7.7,14.0,0.00I
4,948700,5/01/2017,24,62.3,7.0,13.0,0.00I


In [22]:
# inspecting the datatypes of the attributes to decide on 
climate.dtypes

station                      int64
date                        object
air_temperature_celcius      int64
relative_humidity          float64
windspeed_knots            float64
max_wind_speed             float64
precipitation               object
dtype: object

In [23]:
# we noticed that the date attributes needs conversion to datetime object.

# we import dateutil.parser to convert the date attribute to a datetime object.

date = []
for i in climate.iterrows():
    date.append(dateutil.parser.parse(str(i[1].date)))

climate["date"] = date

In [24]:
# inspecting the fire data set.
fire.head(5)

,latitude,longitude,datetime,confidence,date,surface_temperature_celcius
0,-37.966,145.051,2017-12-27T04:16:51,78,27/12/2017,68
1,-35.541,143.311,2017-12-27T00:02:15,82,27/12/2017,63
2,-35.554,143.307,2017-12-27T00:02:15,67,27/12/2017,53
3,-35.543,143.316,2017-12-27T00:02:14,86,27/12/2017,67
4,-37.708,145.100,2017-12-25T04:29:08,80,25/12/2017,54


In [25]:
# inspecting the datatypes of the attributes to decide on 
fire.dtypes

latitude                       float64
longitude                      float64
datetime                        object
confidence                       int64
date                            object
surface_temperature_celcius      int64
dtype: object

In [26]:
# we noticed that the date,datetime attributes needs conversion to datetime object.

# we import dateutil.parser to convert the date attribute to a datetime object.

date_time = []
date = []
for i in fire.iterrows():
    date_time.append(dateutil.parser.parse(i[1].datetime))
    date.append(dateutil.parser.parse(str(i[1].date)))
    
fire["datetime"] = date_time
fire["date"] = date

We will then convert all the fire and climate records into a dictionary format with key-value pairs according to records in a data.
The code to perform the same is as follows:

In [27]:
# convert the records to key-value pair.
climate_dict = climate.to_dict(orient = "records")
fire_dict = fire.to_dict(orient = "records")

### Insering the data into respective collections in database.

We can use `insert_many()` function to insert many records of dictionary or json format into MongoDB. Code to perfome the same is as below:

In [33]:
# insert all the climate historic records to the collection.
climate_collection.insert_many(climate_dict)

In [34]:
# insert all the fire historic records to the collection.
fire_collection.insert_many(fire_dict)

In [30]:
# printing the first document in the climate collection.
for x in climate_collection.find():
    print(x)
    break

{'air_temperature_celcius': 19, 'relative_humidity': 56.8, 'precipitation ': ' 0.00I', '_id': ObjectId('5ce78e0a9343690e9b94dee5'), 'windspeed_knots': 7.9, 'max_wind_speed': 11.1, 'station': 948700, 'date': datetime.datetime(2016, 12, 31, 0, 0)}


In [31]:
# printing the first document in the fire collection.
for y in fire_collection.find():
    print(y)
    break

{'confidence': 78, 'latitude': -37.966, 'datetime': datetime.datetime(2017, 12, 27, 4, 16, 51), '_id': ObjectId('5ce78e0d9343690e9b94e053'), 'longitude': 145.05100000000002, 'surface_temperature_celcius': 68, 'date': datetime.datetime(2017, 12, 27, 0, 0)}


## Task B

## Querying MongoDB using PyMongo

#### A. Find climate data on ​10th December 2017.​

In [35]:
# finding climate data on 10th december

# query statement
myquery = {"date":datetime.strptime("10/12/2017", "%d/%m/%Y")}

# finding the record in the collection.
mydoc = climate_collection.find(myquery)

# printing the  output.
[x for x in mydoc]

[{'_id': ObjectId('5ce78fe39343690e9b94ebdb'),
  'air_temperature_celcius': 15,
  'date': datetime.datetime(2017, 12, 10, 0, 0),
  'max_wind_speed': 14.0,
  'precipitation ': ' 0.00G',
  'relative_humidity': 44.5,
  'station': 948702,
  'windspeed_knots': 10.2}]

#### B. Find the latitude,​ ​longitude, surface temperature (​ °C)​, ​and ​confidence ​when the surface temperature (°C) was between ​65 °C​ and​ 100 °C.​

In [36]:
# query to find surface temp between 65 to 100
query = {"surface_temperature_celcius" : {"$gt" : 65,"$lt":100}}

# finding the record in the collection.
results = fire_collection.find(query,{"latitude": 1, "longitude":1,"confidence": 1,"surface_temperature_celcius":1,
                                    "_id":0}).sort([("surface_temperature_celcius",1)])

# printing the  output.
[x for x in results]

[{'confidence': 90,
  'latitude': -37.624,
  'longitude': 149.314,
  'surface_temperature_celcius': 66},
 {'confidence': 73,
  'latitude': -37.444,
  'longitude': 148.101,
  'surface_temperature_celcius': 66},
 {'confidence': 90,
  'latitude': -36.7032,
  'longitude': 144.0992,
  'surface_temperature_celcius': 66},
 {'confidence': 85,
  'latitude': -36.5023,
  'longitude': 143.6038,
  'surface_temperature_celcius': 66},
 {'confidence': 89,
  'latitude': -36.5116,
  'longitude': 144.562,
  'surface_temperature_celcius': 66},
 {'confidence': 80,
  'latitude': -36.1559,
  'longitude': 141.8102,
  'surface_temperature_celcius': 66},
 {'confidence': 90,
  'latitude': -37.6267,
  'longitude': 142.9993,
  'surface_temperature_celcius': 66},
 {'confidence': 90,
  'latitude': -36.6828,
  'longitude': 144.784,
  'surface_temperature_celcius': 66},
 {'confidence': 89,
  'latitude': -37.4843,
  'longitude': 143.0592,
  'surface_temperature_celcius': 66},
 {'confidence': 84,
  'latitude': -38.0423,

#### C. Find ​date,​ ​surface temperature (°C), air temperature (°C), relative humidity and max wind speed on​ 15th and 16th of December 2017.

In [37]:
# perform aggrigation and find the records for the given condition.

# NOTE: We have decided not to unwind the result, so we dont make the output redundant.
result = mydb.climate.aggregate([{
    "$lookup":
    {
        "from": "fire",
        "localField": "date",
        "foreignField": "date",
        "as":"fire",
    }},
    {
    "$match" : {"$or":[{"date": datetime.strptime("15/12/2017", "%d/%m/%Y")},{"date":datetime.strptime("16/12/2017", "%d/%m/%Y")}]}

    },
    {
    "$project" : {"_id":0,"date":1,"air_temperature_celcius":1,"relative_humidity":1,"max_wind_speed":1,"fire":{"surface_temperature_celcius":1}}  
    }
    ])

In [38]:
# printing the  output.
[x for x in result]

[{'air_temperature_celcius': 18,
  'date': datetime.datetime(2017, 12, 15, 0, 0),
  'fire': [{'surface_temperature_celcius': 42},
   {'surface_temperature_celcius': 36},
   {'surface_temperature_celcius': 38},
   {'surface_temperature_celcius': 40}],
  'max_wind_speed': 14.0,
  'relative_humidity': 52.0},
 {'air_temperature_celcius': 18,
  'date': datetime.datetime(2017, 12, 16, 0, 0),
  'fire': [{'surface_temperature_celcius': 43},
   {'surface_temperature_celcius': 33},
   {'surface_temperature_celcius': 54},
   {'surface_temperature_celcius': 73},
   {'surface_temperature_celcius': 55},
   {'surface_temperature_celcius': 75},
   {'surface_temperature_celcius': 55},
   {'surface_temperature_celcius': 66},
   {'surface_temperature_celcius': 56},
   {'surface_temperature_celcius': 60},
   {'surface_temperature_celcius': 73},
   {'surface_temperature_celcius': 48},
   {'surface_temperature_celcius': 55},
   {'surface_temperature_celcius': 64},
   {'surface_temperature_celcius': 57}],
  

#### D. Find ​datetime, air temperature (°C), surface temperature (°C) and ​confidence when the ​confidence i​ s between 80 and 100.

In [39]:
# perform aggrigation and find the records for confidence between 80 and 100.

result = mydb.fire.aggregate([{
    "$lookup":
    {
        "from": "climate",
        "localField": "date",
        "foreignField": "date",
        "as":"climate",
    }},
    {
    "$project" : {"_id":0,"datetime": 1,
                "surface_temperature_celcius":1,"confidence":1,
                "climate":{"air_temperature_celcius":1}}  
    },
    {
    "$match" : {"confidence":{"$gt": 80, "$lt": 100}}
    }
    ])

In [40]:
# printing the  output.
[x for x in result]

[{'climate': [{'air_temperature_celcius': 28}],
  'confidence': 82,
  'datetime': datetime.datetime(2017, 12, 27, 0, 2, 15),
  'surface_temperature_celcius': 63},
 {'climate': [{'air_temperature_celcius': 28}],
  'confidence': 86,
  'datetime': datetime.datetime(2017, 12, 27, 0, 2, 14),
  'surface_temperature_celcius': 67},
 {'climate': [{'air_temperature_celcius': 18}],
  'confidence': 94,
  'datetime': datetime.datetime(2017, 12, 16, 15, 38, 39),
  'surface_temperature_celcius': 43},
 {'climate': [{'air_temperature_celcius': 18}],
  'confidence': 93,
  'datetime': datetime.datetime(2017, 12, 16, 4, 35, 13),
  'surface_temperature_celcius': 73},
 {'climate': [{'air_temperature_celcius': 18}],
  'confidence': 84,
  'datetime': datetime.datetime(2017, 12, 16, 4, 34, 58),
  'surface_temperature_celcius': 55},
 {'climate': [{'air_temperature_celcius': 18}],
  'confidence': 95,
  'datetime': datetime.datetime(2017, 12, 16, 4, 34, 58),
  'surface_temperature_celcius': 75},
 {'climate': [{'a

#### E. Find the top 10 records with the highest ​surface temperature​ ​(°C).

In [41]:
#  query to aggregate top 10 records w.r.t surface temperature
results = fire_collection.aggregate([{"$sort": {"surface_temperature_celcius":-1}},{"$limit": 10}])

# print the output
[x for x in results]

[{'_id': ObjectId('5ce78fe69343690e9b94f0d0'),
  'confidence': 100,
  'date': datetime.datetime(2017, 4, 18, 0, 0),
  'datetime': datetime.datetime(2017, 4, 18, 4, 52),
  'latitude': -38.1665,
  'longitude': 143.062,
  'surface_temperature_celcius': 124},
 {'_id': ObjectId('5ce78fe69343690e9b94f57a'),
  'confidence': 100,
  'date': datetime.datetime(2017, 4, 4, 0, 0),
  'datetime': datetime.datetime(2017, 4, 4, 4, 32, 50),
  'latitude': -36.343,
  'longitude': 142.1986,
  'surface_temperature_celcius': 123},
 {'_id': ObjectId('5ce78fe69343690e9b94f042'),
  'confidence': 100,
  'date': datetime.datetime(2017, 1, 5, 0, 0),
  'datetime': datetime.datetime(2017, 5, 1, 4, 14, 20),
  'latitude': -36.9318,
  'longitude': 143.0907,
  'surface_temperature_celcius': 122},
 {'_id': ObjectId('5ce78fe69343690e9b94f669'),
  'confidence': 100,
  'date': datetime.datetime(2017, 3, 18, 0, 0),
  'datetime': datetime.datetime(2017, 3, 18, 3, 50, 50),
  'latitude': -37.016999999999996,
  'longitude': 148.

#### F. Find the number of fire in each day. You are required to only display ​the total number of fire​ and ​the date​ in the output.

In [42]:
# We can determine number of fire in each day
result = fire_collection.aggregate([
    {"$group" : {"_id":"$date", "Number of fires":{"$sum":1}}},
    { "$project" : { "_id" : 0, "date": "$_id", "Number of fires":1 } } 
])

# print the output
[x for x in result] 

[{'Number of fires': 2, 'date': datetime.datetime(2017, 6, 3, 0, 0)},
 {'Number of fires': 2, 'date': datetime.datetime(2017, 8, 3, 0, 0)},
 {'Number of fires': 3, 'date': datetime.datetime(2017, 9, 3, 0, 0)},
 {'Number of fires': 2, 'date': datetime.datetime(2017, 3, 13, 0, 0)},
 {'Number of fires': 7, 'date': datetime.datetime(2017, 3, 15, 0, 0)},
 {'Number of fires': 6, 'date': datetime.datetime(2017, 3, 17, 0, 0)},
 {'Number of fires': 21, 'date': datetime.datetime(2017, 3, 19, 0, 0)},
 {'Number of fires': 54, 'date': datetime.datetime(2017, 3, 28, 0, 0)},
 {'Number of fires': 5, 'date': datetime.datetime(2017, 2, 4, 0, 0)},
 {'Number of fires': 49, 'date': datetime.datetime(2017, 5, 4, 0, 0)},
 {'Number of fires': 20, 'date': datetime.datetime(2017, 8, 4, 0, 0)},
 {'Number of fires': 2, 'date': datetime.datetime(2017, 3, 24, 0, 0)},
 {'Number of fires': 24, 'date': datetime.datetime(2017, 11, 4, 0, 0)},
 {'Number of fires': 69, 'date': datetime.datetime(2017, 12, 4, 0, 0)},
 {'Num

#### G.Find the ​average surface temperature ​(°C) f​ or each day​. You are required to only display ​average surface temperature (°C)​ ​and ​the date​ in the output.

In [43]:
# Determine Average surface temperature (°C) for each day

result = fire_collection.aggregate([{"$group": {"_id":"$date", "average_surface_temp": {"$avg":"$surface_temperature_celcius"} } },
                                 { "$project" : { "_id" : 0, "date": "$_id", "average_surface_temp":1 } }])

# print the output
[x for x in result] 

[{'average_surface_temp': 60.5, 'date': datetime.datetime(2017, 6, 3, 0, 0)},
 {'average_surface_temp': 51.5, 'date': datetime.datetime(2017, 8, 3, 0, 0)},
 {'average_surface_temp': 46.666666666666664,
  'date': datetime.datetime(2017, 9, 3, 0, 0)},
 {'average_surface_temp': 38.5, 'date': datetime.datetime(2017, 3, 13, 0, 0)},
 {'average_surface_temp': 46.0, 'date': datetime.datetime(2017, 3, 15, 0, 0)},
 {'average_surface_temp': 59.5, 'date': datetime.datetime(2017, 3, 17, 0, 0)},
 {'average_surface_temp': 65.57142857142857,
  'date': datetime.datetime(2017, 3, 19, 0, 0)},
 {'average_surface_temp': 60.925925925925924,
  'date': datetime.datetime(2017, 3, 28, 0, 0)},
 {'average_surface_temp': 45.2, 'date': datetime.datetime(2017, 2, 4, 0, 0)},
 {'average_surface_temp': 53.142857142857146,
  'date': datetime.datetime(2017, 5, 4, 0, 0)},
 {'average_surface_temp': 60.75, 'date': datetime.datetime(2017, 8, 4, 0, 0)},
 {'average_surface_temp': 49.0, 'date': datetime.datetime(2017, 3, 24, 0,